In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-toy-dataset/covid_toy.csv


In [2]:
df = pd.read_csv("/kaggle/input/covid-toy-dataset/covid_toy.csv")

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isnull().any()

age          False
gender       False
fever         True
cough        False
city         False
has_covid    False
dtype: bool

Here we need <br>
fever - Missing Values <br>
gender, city - Nominal Categorial Data <br>
cough - Ordinal Categorical Data <br>
has_covid - Label Encoder <br>

<p>If we do this manually one by one, its very hectic <br>
Column Trnasformer makes our life easier</p>

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
X_train.head()

,age,gender,fever,cough,city
77,8,Female,101.0,Mild,Kolkata
5,84,Female,NaN,Mild,Bangalore
30,15,Male,101.0,Mild,Delhi
13,64,Male,102.0,Mild,Bangalore
83,17,Female,104.0,Mild,Kolkata


In [7]:
y_train.head()

77     No
5     Yes
30    Yes
13    Yes
83     No
Name: has_covid, dtype: object

<h2>Without Column Transformer</h2>

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [10]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [11]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [12]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [13]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [28]:
X_train_final_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_final_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

<h2>Using Column Transformer!!</h2>

In [33]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('imp',SimpleImputer(),['fever']),
    ('ode',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('ohe',OneHotEncoder(sparse=False,drop='first'),['gender','city']),
],remainder='passthrough')

X_train_coltrans = transformer.fit_transform(X_train)
X_test_coltrans = transformer.transform(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [31]:
print(X_train_coltrans.shape)
print(X_train_final_transformed.shape)
print(X_test_coltrans.shape)
print(X_test_final_transformed.shape)

(80, 7)
(80, 7)
(20, 7)
(20, 7)


In [32]:
print(X_train_coltrans[0])
print(X_train_final_transformed[0])

[101.   0.   0.   0.   1.   0.   8.]
[  8. 101.   0.   0.   1.   0.   0.]
